# [고급 Numpy]

In [1]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
np.set_printoptions(precision=4)
import sys

%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## 12.1: ndarray 객체 내부 알아보기(ndarray object internals)
- [ndarray의 내부적 포함요소]
    - 데이터에 대한 포인터, 시스템 메모리를 가리킴
    - 자료형 또는 dtype
    - 배열의 형태를 나타내는 튜플, 예를들어 10 * 5 배열은 (10,5)와 같은 형태를 취한다.

In [6]:
np.ones((10,5)).shape

(10, 5)

In [8]:
# 스트라이드 튜플(strides) : 하나의 원소에서 다음 원소까지의 너비를 표현한 정수 튜플.
np.ones((3,4,5), dtype=np.float64).strides

(160, 40, 8)

### 12.1.1: Numpy dtype 구조(NumPy dtype hierarchy)
- dtype은 np.integer.np.floating 같은 상위클래스를 가지므로 np.issubdtype 함수와 함께 사용가능

In [9]:
ints = np.ones(10, dtype=np.uint16)
floats = np.ones(10, dtype=np.float32)
np.issubdtype(ints.dtype, np.integer)
np.issubdtype(floats.dtype, np.floating)

True

True

In [11]:
# 각 자료형의 mro 메서드를 이용해 dtype의 모든 부모클래스 목록을 확인가능.
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

## 12.2: 고급 배열 조작 기법(Advanced array manipulation)

### 12.2.1: 배열 재형성하기(Reshaping arrays)
- 배열의 모양을 변환하려면 배열의 인스턴스 메서드인 reshape메서드에 새로운 모양을 나타내는 튜플을 넘김.

In [4]:
arr = np.arange(8)
arr
arr.reshape((4,2))

array([0, 1, 2, 3, 4, 5, 6, 7])

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

In [6]:
arr.reshape((4,2)).reshape((2,4))

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

In [11]:
arr = np.arange(15)
arr.reshape((5,-1))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [17]:
other_arr = np.ones((3,5))
other_arr.shape
arr.reshape(other_arr.shape) # shape 속성은 튜플이라서 reshape 메서드에 직접 넘기는 것도 가능.

(3, 5)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [23]:
# 평탄화 : 다차원 배열을 낮은 차원으로 변환.
arr = np.arange(15).reshape((5,3))
arr
# ravel : 평탄화 시키는 메서드, 원본데이터의 복사본을 생성하지 않음
# flatten : ravel와 똑같은 결과, 항상 데이터의 복사본을 반환.
arr.ravel()
arr.flatten()

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

### 12.2.2: C와 포트란 순서(C vs. Fortran order)
- row, column의 우선운위 : C순서 -> 포트란순서
- reshape, ravel 같은 함수는 배열에서 데이터 순서를 나타내는 인자를 받음
    - 'C', 'F'인데 드물게 'A', 'K'를 쓰기도 함.
- C(row 우선순위) : 상위차원을 먼저 탐색.(1번축이 0번축보다 우선 탐색)
- 포트란(column 우선순위) : 상위차원을 나중에 탐색.(0번축이 1번축보다 우선 탐색)

In [34]:
arr = np.arange(12).reshape((3,4))
arr
arr.ravel()
arr.ravel('F')

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

### 12.2.3: 배열 이어붙이고 나누기(Concatenating and splitting arrays)
- numpy.concatenate : 배열의 목록(튜플, 리스트 등)을 받아서 주어진 axis에 따라 하나의 배열로 합쳐준다.
- [배열 이어붙이기 함수]
    - concatenate : 대표함수, 하나의 축을 따라 배열을 이어붙임
    - vstack, row_stack : 로우(axis=0)을 따라 배열을 쌓음.
    - hstack : 컬럼(axis=1)을 따라 배열을 쌓음.
    - column_stack : hstack과 같지만 1차원 배열을 2차원 컬럼벡터로 먼저 변환.
    - dstack : 깊이(axis=2)에 따라 배열을 쌓음.
    - split : 특정 축을 따라 지정된 위치를 기점으로 배열을 나눈다.
    - hsplit, vsplit, dsplit : 각각 axis 0,1,2를 배열을 나누는 함수.

In [37]:
arr1 = np.array([[1,2,3],[4,5,6]])
arr2 = np.array([[7,8,9],[10,11,12]])
np.concatenate([arr1, arr2], axis=0) # 행기준
np.concatenate([arr1, arr2], axis=1) # 열기준

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

In [39]:
# vstack, hstack 함수 : 배열 이어붙이기 작업을 쉽게 처리.
np.vstack((arr1, arr2))
np.hstack((arr1, arr2))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

In [41]:
from numpy.random import randn
arr = randn(5, 2)
arr
first, second, third = np.split(arr, [1, 3])
first
second
third

array([[ 0.2802,  0.9519],
       [ 2.5329,  1.3597],
       [-1.542 ,  0.1671],
       [ 0.1071, -0.7458],
       [ 0.3386, -2.2775]])

array([[ 0.2802,  0.9519]])

array([[ 2.5329,  1.3597],
       [-1.542 ,  0.1671]])

array([[ 0.1071, -0.7458],
       [ 0.3386, -2.2775]])

#### 배열쌓기도우미(Stacking helpers): r_ & c_

In [44]:
arr = np.arange(6)
arr1 = arr.reshape((3,2))
arr2 = randn(3,2)
np.r_[arr1, arr2]
np.c_[np.r_[arr1, arr2], arr]

array([[ 0.    ,  1.    ],
       [ 2.    ,  3.    ],
       [ 4.    ,  5.    ],
       [-0.3675, -0.0073],
       [ 0.2832, -0.4293],
       [-0.2344, -0.5367]])

array([[ 0.    ,  1.    ,  0.    ],
       [ 2.    ,  3.    ,  1.    ],
       [ 4.    ,  5.    ,  2.    ],
       [-0.3675, -0.0073,  3.    ],
       [ 0.2832, -0.4293,  4.    ],
       [-0.2344, -0.5367,  5.    ]])

In [46]:
np.c_[1:6, -10:-5]

array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

### 12.2.4: 원소 반복시키기: repeat과 tile(Repeating elements: tile and repeat)
- repeat, tile : 큰 배열을 만들기 위해 배열을 반복하거나 복제하는 함수.

In [49]:
arr = np.arange(3)
arr.repeat(4)

array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2])

In [51]:
arr.repeat([2,3,4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

In [54]:
arr = randn(2,2)
arr
arr.repeat(2, axis=0)

array([[-0.6693, -0.9731],
       [-1.148 , -0.4148]])

array([[-0.6693, -0.9731],
       [-0.6693, -0.9731],
       [-1.148 , -0.4148],
       [-1.148 , -0.4148]])

In [56]:
arr.repeat([2, 3], axis=0)
arr.repeat([2, 3], axis=1)

array([[-0.6693, -0.9731],
       [-0.6693, -0.9731],
       [-1.148 , -0.4148],
       [-1.148 , -0.4148],
       [-1.148 , -0.4148]])

array([[-0.6693, -0.6693, -0.9731, -0.9731, -0.9731],
       [-1.148 , -1.148 , -0.4148, -0.4148, -0.4148]])

In [64]:
# tile : 벽에 타일을 이어붙이듯 내용을 붙임.
arr
np.tile(arr, 3)

array([[-0.6693, -0.9731],
       [-1.148 , -0.4148]])

array([[-0.6693, -0.9731, -0.6693, -0.9731, -0.6693, -0.9731],
       [-1.148 , -0.4148, -1.148 , -0.4148, -1.148 , -0.4148]])

In [63]:
arr
np.tile(arr, (2, 1))
np.tile(arr, (3, 2))

array([[-0.6693, -0.9731],
       [-1.148 , -0.4148]])

array([[-0.6693, -0.9731],
       [-1.148 , -0.4148],
       [-0.6693, -0.9731],
       [-1.148 , -0.4148]])

array([[-0.6693, -0.9731, -0.6693, -0.9731],
       [-1.148 , -0.4148, -1.148 , -0.4148],
       [-0.6693, -0.9731, -0.6693, -0.9731],
       [-1.148 , -0.4148, -1.148 , -0.4148],
       [-0.6693, -0.9731, -0.6693, -0.9731],
       [-1.148 , -0.4148, -1.148 , -0.4148]])

### 12.2.5: 팬시 색인: take와 put(Fancy indexing equivalents: take and put)
- 정수배열을 사용한 "팬시색인" 기능을 통해 배열의 일부값을 지정하거나 가져올 수 있다.

In [66]:
arr = np.arange(10) * 100
inds = [7,1,2,6]
arr[inds]

array([700, 100, 200, 600])

In [75]:
# take : ndarray에는 단일축에 대한 값을 선택할때만 사용할 수 있는 유용한 메서드 존재.
arr.take(inds)
arr.put(inds, 42)
arr
arr.put(inds, [40,41,42,43])
arr

array([40, 41, 42, 43])

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

In [80]:
inds = [2,0,2,1]
arr = randn(2,4)
arr
arr.take(inds, axis=1)

array([[-1.0087,  1.267 ,  0.0149,  0.1543],
       [-0.7605,  0.6045,  0.3404, -0.6963]])

array([[ 0.0149, -1.0087,  0.0149,  1.267 ],
       [ 0.3404, -0.7605,  0.3404,  0.6045]])

In [84]:
# note : take와 팬시색인 시간적 차이
arr = randn(1000,50)

# 500개 로우를 가지는 무작위 표본
inds = np.random.permutation(1000)[:500]

%timeit arr[inds]
%timeit arr.take(inds, axis=0)

10000 loops, best of 3: 109 µs per loop
The slowest run took 4.60 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 89.2 µs per loop


## 12.3: 브로드캐스팅(Broadcasting)
- 브로드캐스팅 : 다른모양의 배열 간 산술연산을 어떻게 수행해야 하는지 설명.
- [브로드캐스팅의 규칙]
    - 이어지는 각 차원(시작부터 끝까지)에 대해 축의 길이가 일치하거나 둘 중 하나의 길이가 1이라면 두 배열은 브로드캐스팅 호환. 브로드캐스팅은 누락된, 혹은 길이가 1인 차원에 대해 수행.

In [92]:
arr = np.arange(5)
arr
arr * 4 # 스칼라값 4는 곱셈연산과정에서 배열의 모든원소로 전파(broadcast)

array([0, 1, 2, 3, 4])

array([ 0,  4,  8, 12, 16])

In [94]:
arr = randn(4,3)
arr.mean(0)
demeaned = arr - arr.mean(0)
demeaned
demeaned.mean(0)

array([ 0.3031, -0.0284,  0.2838])

array([[-0.674 , -0.3699, -0.2988],
       [-0.6034,  0.8134,  0.2861],
       [ 0.3007, -0.7504,  0.0722],
       [ 0.9768,  0.3069, -0.0595]])

array([ -5.5511e-17,   2.7756e-17,  -3.4694e-17])

In [97]:
arr
row_means = arr.mean(1)
row_means
row_means.reshape((4, 1))
demeaned = arr - row_means.reshape((4, 1))
demeaned.mean(1)

array([[-0.3709, -0.3983, -0.015 ],
       [-0.3003,  0.785 ,  0.5699],
       [ 0.6038, -0.7788,  0.3561],
       [ 1.2799,  0.2784,  0.2244]])

array([-0.2614,  0.3515,  0.0603,  0.5942])

array([[-0.2614],
       [ 0.3515],
       [ 0.0603],
       [ 0.5942]])

array([ -1.8504e-17,   0.0000e+00,  -1.8504e-17,   3.7007e-17])

### 12.3.1: 다른축에 대해 브로드캐스팅하기(Broadcasting over other axes)

In [99]:
arr - arr.mean(1) # 0번축이 아닌 다른축에 대해 산술연산 수행 > 축의 데이터개수가 안맞음
# (4,1)임

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

In [101]:
arr - arr.mean(1).reshape((4, 1))

array([[-0.1095, -0.1369,  0.2464],
       [-0.6518,  0.4334,  0.2184],
       [ 0.5434, -0.8392,  0.2957],
       [ 0.6857, -0.3158, -0.3699]])

In [103]:
# np.newaxis : 배열을 전체 슬라이스와 함께 사용해 새로운 축을 추가하는 함수.
arr = np.zeros((4, 4))
arr_3d = arr[:, np.newaxis, :]
arr_3d.shape

(4, 1, 4)

In [106]:
arr_1d = np.random.normal(size=3)
arr_1d[:, np.newaxis]
arr_1d[np.newaxis, :]

array([[-0.5289],
       [ 0.5314],
       [-0.5398]])

array([[-0.5289,  0.5314, -0.5398]])

In [108]:
arr = randn(3, 4, 5)
depth_means = arr.mean(2)
depth_means
demeaned = arr - depth_means[:, :, np.newaxis]
demeaned.mean(2)

array([[ 0.3718, -0.0341, -0.8979, -0.6929],
       [ 0.1041, -0.2068, -0.5153,  0.7858],
       [ 0.6401, -0.1698, -0.2184,  0.2884]])

array([[ -2.2204e-17,   8.3267e-18,   1.3323e-16,   4.4409e-17],
       [  2.2204e-17,  -1.1102e-17,  -1.3323e-16,   0.0000e+00],
       [  8.8818e-17,   0.0000e+00,   1.1102e-17,  -4.4409e-17]])

In [111]:
# 평균값을 빼는 과정을 일반화하는 함수.
def demean_axis(arr, axis=0):
    means = arr.mean(axis)
    # [:, :, np.newaxis] : 같은 거슬 n차원으로 일반화시킨다.
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[indexer]

### 12.3.2: 브로드캐스팅 이용해 배열에 값 대입하기 (Setting array values by broadcasting)

In [113]:
arr = np.zeros((4, 3))
arr[:] = 5
arr

array([[ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.]])

In [115]:
col = np.array([1.28, -0.42, 0.44, 1.6])
arr[:] = col[:, np.newaxis]
arr
arr[:2] = [[-1.37], [0.509]]
arr

array([[ 1.28,  1.28,  1.28],
       [-0.42, -0.42, -0.42],
       [ 0.44,  0.44,  0.44],
       [ 1.6 ,  1.6 ,  1.6 ]])

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509],
       [ 0.44 ,  0.44 ,  0.44 ],
       [ 1.6  ,  1.6  ,  1.6  ]])

## 12.4: 고급 ufunc 사용법(Advanced ufunc usage)

### 12.4.1: ufunc 인스턴스 메서드(Ufunc instance methods)

In [117]:
# reduce : 하나의 배열을 받아 순차적인 이항연산을 통해 축에 따라 그 값을 집계.
arr = np.arange(10)
np.add.reduce(arr) # 배열의 모든 원소를 더하는 방법.
arr.sum()

45

45

In [119]:
np.random.seed(12346)

In [125]:
# (예제) np.logical_and 는 all과 동일
arr = randn(5, 5)
arr[::2].sort(1) # sort a few rows
arr
arr[:, :-1]
arr[:, 1:]
arr[:, :-1] < arr[:, 1:]
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis=1)

array([[-0.273 ,  0.6865,  0.7168,  0.7372,  1.6286],
       [ 0.9352, -0.721 ,  1.0755, -0.3052, -0.2362],
       [-0.7954, -0.2982,  0.5234,  0.6525,  1.0092],
       [ 0.0252,  0.5525, -0.8539, -0.6434,  0.6691],
       [-2.2769, -1.5059, -1.2337, -0.5629,  0.4805]])

array([[-0.273 ,  0.6865,  0.7168,  0.7372],
       [ 0.9352, -0.721 ,  1.0755, -0.3052],
       [-0.7954, -0.2982,  0.5234,  0.6525],
       [ 0.0252,  0.5525, -0.8539, -0.6434],
       [-2.2769, -1.5059, -1.2337, -0.5629]])

array([[ 0.6865,  0.7168,  0.7372,  1.6286],
       [-0.721 ,  1.0755, -0.3052, -0.2362],
       [-0.2982,  0.5234,  0.6525,  1.0092],
       [ 0.5525, -0.8539, -0.6434,  0.6691],
       [-1.5059, -1.2337, -0.5629,  0.4805]])

array([[ True,  True,  True,  True],
       [False,  True, False,  True],
       [ True,  True,  True,  True],
       [ True, False,  True,  True],
       [ True,  True,  True,  True]], dtype=bool)

array([ True, False,  True, False,  True], dtype=bool)

In [130]:
# accumulate : 누계를 담고 있는 같은 크기의 배열을 생성.
arr = np.arange(15).reshape((3,5))
arr
np.add.accumulate(arr, axis=1)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]], dtype=int32)

In [133]:
# outer : 두 배열간의 벡터 곱, 외적을 계산
arr = np.arange(3).repeat([1,2,2])
arr
np.multiply.outer(arr, np.arange(5))

array([0, 1, 1, 2, 2])

array([[0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 6, 8],
       [0, 2, 4, 6, 8]])

In [136]:
# outer 메서드 결과의 차원은 입력된 차원의 합과 같다.
result = np.subtract.outer(randn(3,4), randn(5))
result.shape

(3, 4, 5)

In [141]:
# reduceat 
# : 로컬 reduce를 수행하는데, 본질적으로 배열의 groupby 연산으로 배열의 슬라이스를 모두 함께 집계한 것.
# : 값을 어떻게 나누고 집계할지 나타내는 경계목록을 인자로 받음.
arr = np.arange(10)
arr
np.add.reduceat(arr, [0,5,8])
# arr[0:5], arr[5:8], arr[8:] 의 합 결과.

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

array([10, 18, 17], dtype=int32)

In [144]:
arr = np.multiply.outer(np.arange(4), np.arange(5))
arr
np.add.reduceat(arr, [0,2,4], axis=1)

array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12]])

array([[ 0,  0,  0],
       [ 1,  5,  4],
       [ 2, 10,  8],
       [ 3, 15, 12]], dtype=int32)

### 12.4.2: 사용자 ufunc(Custom ufuncs)
- numpy.frompyfunc : 입출력에 대한 표준과 함께 파이썬 함수를 인자로 취하는 사용자함수로 만들수 있음.

In [145]:
def add_elements(x, y):
    return x + y;
add_them = np.frompyfunc(add_elements, 2,1)
add_them(np.arange(8), np.arange(8))
# np.frompyfunc를 이용해서 생성한 함수는 항상 파이썬 객체가 담긴 배열을 반환 > 안좋음
# 대안 : np.vectorize로 자료형을 추론.

array([0, 2, 4, 6, 8, 10, 12, 14], dtype=object)

In [147]:
# np.vectorize 자료형 추론.
add_them = np.vectorize(add_elements, otypes=[np.float64])
add_them(np.arange(8), np.arange(8))

array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.])

In [150]:
# ufunc 스타일의 함수는 각 원소를 계산하기 위해 파이썬 함수를 호출하므로, Numpy의 C기반 ufunc반복문보다 느리다.
arr = randn(10000)
%timeit add_them(arr, arr)
%timeit np.add(arr, arr)

100 loops, best of 3: 11 ms per loop
The slowest run took 18.91 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 20.3 µs per loop


## 12.5: 구조화된 배열과 레코드 배열(Structured and record arrays)
- 구조화된 배열 : 배열의 각 원소가 C의 구조체 혹은 다양한 이름의 필드를 갖는 SQL테이블의 한 로우라고 생각할 수 있는 ndarray이다.

In [152]:
dtype = [('x', np.float64), ('y', np.int32)]
sarr = np.array([(1.5, 6),(np.pi, -2)], dtype=dtype)
sarr

array([( 1.5   ,  6), ( 3.1416, -2)],
      dtype=[('x', '<f8'), ('y', '<i4')])

In [154]:
sarr[0]
sarr[0]['y']

( 1.5, 6)

6

In [155]:
sarr['x']

array([ 1.5   ,  3.1416])

### 12.5.1: 중첩된 dtype과 다차원 필드(Nested dtypes and multidimensional fields)

In [157]:
# 구조화된 dtype을 지정할 때 추가적으로 그 모양(정수나 튜플)을 전달 할 수 있음.
dtype = [('x', np.int64, 3), ('y', np.int32)]
arr = np.zeros(4, dtype=dtype)
arr

array([([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0)],
      dtype=[('x', '<i8', (3,)), ('y', '<i4')])

In [159]:
arr[0]['x']

array([0, 0, 0], dtype=int64)

In [161]:
arr['x']

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int64)

In [167]:
# 복잡한 중첩구조로 하나의 배열 안에서 단일 메모리로 표현할 수 있음.
# but, pandas의 DataFrame의 계층적 색인이 이와 유사하긴 하지만 이런 기능을 지원하지 않음.
dtype = [('x',[('a','f8'),('b', 'f4')]), ('y', np.int32)]
data = np.array([((1,2),5), ((3,4),6)], dtype=dtype)
data['x']
data['y']
data['x']['b']

array([( 1.,  2.), ( 3.,  4.)],
      dtype=[('a', '<f8'), ('b', '<f4')])

array([5, 6])

array([ 2.,  4.], dtype=float32)

### 12.5.2: 구조화된 배열을 사용해야 하는 이유(Why use structured arrays?)
- 데이터를 디스크에서 읽거나 쓰고 네트워크를 통해 전송할 때 매우 빠르고 효과적인 방법을 제공하기 때문.

### 12.5.3: 구조화된 배열 다루기:numpy.lib.recfunctions(Structured array manipulations: numpy.lib.recfunctions)
- numpy.lib.recfunctions 라이브러리 참고 : 필드를 추가/삭제, 기본적인 조인과 유사한 연산 수행 등.

## 12.6: 정렬에 관하여(More about sorting)

In [6]:
# 파이썬의 ndarray sort 인스턴스 메서드는 새로운 배열을 직접생성하지 않고 배열의 내용을 결정.
arr = randn(6)
arr.sort()
arr

array([-0.8297, -0.4818,  0.211 ,  0.2409,  0.3082,  0.4452])

In [10]:
# 배열을 그대로 정렬할때는 그 배열이 다른 ndarray의 뷰일 경우 원본배열의 값이 변경됨.
arr = randn(3,5)
arr
arr[:, 0].sort()
arr

array([[ 0.6755, -0.2988, -0.7539, -1.0051,  0.2877],
       [-0.6243, -1.2929, -0.59  ,  0.7433, -0.8668],
       [-0.1634,  0.6623, -1.3627,  0.7384,  0.2649]])

array([[-0.6243, -0.2988, -0.7539, -1.0051,  0.2877],
       [-0.1634, -1.2929, -0.59  ,  0.7433, -0.8668],
       [ 0.6755,  0.6623, -1.3627,  0.7384,  0.2649]])

In [12]:
# numpy.sort : 정렬된 배열의복사본 생성
arr = randn(5)
arr
np.sort(arr)
arr

array([ 0.8487, -0.4552,  0.1703,  0.0591, -0.2768])

array([-0.4552, -0.2768,  0.0591,  0.1703,  0.8487])

array([ 0.8487, -0.4552,  0.1703,  0.0591, -0.2768])

In [14]:
arr = randn(3,5)
arr
arr.sort(axis=1)
arr

array([[-0.7298,  0.2644,  1.0287, -0.4608, -0.625 ],
       [-1.3048,  0.1228, -1.1596,  0.1315,  0.2874],
       [-0.2684,  0.4882, -0.5314, -1.555 ,  1.5537]])

array([[-0.7298, -0.625 , -0.4608,  0.2644,  1.0287],
       [-1.3048, -1.1596,  0.1228,  0.1315,  0.2874],
       [-1.555 , -0.5314, -0.2684,  0.4882,  1.5537]])

In [16]:
arr[:, ::-1] # 뒤집어진 순서.

array([[ 1.0287,  0.2644, -0.4608, -0.625 , -0.7298],
       [ 0.2874,  0.1315,  0.1228, -1.1596, -1.3048],
       [ 1.5537,  0.4882, -0.2684, -0.5314, -1.555 ]])

### 12.6.1: 간접 정렬: argsort과 lexsort(Indirect sorts: argsort and lexsort)
- 데이터 분석시 하나 이상의 키를 기준으로 데이터를 정렬시.
- indexer : 색인을 돌려준다.

In [20]:
values = np.array([5,0,1,3,2])
indexer = values.argsort()
indexer
values[indexer]

array([1, 2, 4, 3, 0], dtype=int64)

array([0, 1, 2, 3, 5])

In [23]:
# 2차월 배열을 첫번째 로우 순서대로 정렬하는 코드.
arr = randn(3,5)
arr[0] = values
arr
arr[:, arr[0].argsort()]

array([[ 5.    ,  0.    ,  1.    ,  3.    ,  2.    ],
       [ 1.9611,  0.1936, -0.035 , -2.1442,  0.9872],
       [-2.372 , -0.3918, -0.7508, -1.5648,  0.4591]])

array([[ 0.    ,  1.    ,  2.    ,  3.    ,  5.    ],
       [ 0.1936, -0.035 ,  0.9872, -2.1442,  1.9611],
       [-0.3918, -0.7508,  0.4591, -1.5648, -2.372 ]])

In [26]:
# numpy.lexsort : argsort와 유사하지만 다중 키 배열에 간접 사전 순으로 정렬.
# 성과 이름으로 구분됨.
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])
sorter = np.lexsort((first_name, last_name))
zip(last_name[sorter], first_name[sorter])

### 12.6.2: 다른 정렬 알고리즘(Alternate sort algorithms)
- 견고한(stable) 정렬 알고리즘 : 동일 원소의 상대적인 위치를 그대로 둔다 > 상대적인 순서가 의미를 가지는 간접정렬의 중요한 기능!

In [6]:
values = np.array(['2:first','2:second','1:first','1:second','1:third'])
key = np.array([2,2,1,1,1])
indexer = key.argsort(kind='mergesort') # mergesort : 시간복잡도 O(nlog(n))
indexer
values.take(indexer)

array([2, 3, 4, 0, 1], dtype=int64)

array(['1:first', '1:second', '1:third', '2:first', '2:second'],
      dtype='<U8')

- quicksort : 속도 1, 견고함 no , 공간복잡도 0, 시간복잡도 $O(n^2)$
- mergesort : 속도 2, 견고함 yes, 공간복잡도 n/2, 시간복잡도 $O(nlog(n))$
- heapsort : 속도 3, 견고함 no , 공간복잡도 0, 시간복잡도 $O(nlog(n))$

### 12.6.3: numpy.searchsorted: 정렬된 배열에서 원소 찾기(numpy.searchsorted: Finding elements in a sorted array)
- numpy.searchsorted : 정렬된 배열상에서 이진탐색을 수행해 새로운 값을 삽입할때 정렬된 상태를 유지하기 위해 위치를 반환하는 메서드.

In [15]:
arr = np.array([0,1,7,12,15])
arr
arr.searchsorted(5) # 탐색시 가까운 index 위치.

array([ 0,  1,  7, 12, 15])

2

In [16]:
arr.searchsorted([0, 8, 11, 16])

array([0, 3, 3, 5], dtype=int64)

In [18]:
arr = np.array([0,0,0,1,1,1,1])
arr.searchsorted([0,1]) 
arr.searchsorted([0,1], side='right')

array([0, 3], dtype=int64)

array([3, 7], dtype=int64)

In [28]:
# 1~10000 까지의 값을 특정 구간별로 나눈 배열
data = np.floor(np.random.uniform(0,1000, size=50))
bins = np.array([0,100,1000,5000,10000])
data

array([  81.,   55.,  636.,  277.,  833.,  878.,  633.,  685.,    2.,
        735.,  575.,   96.,  979.,  264.,  598.,   86.,   97.,   33.,
        943.,  623.,  533.,  926.,  749.,  133.,  884.,  738.,  697.,
        209.,  354.,  559.,  957.,  222.,   32.,  186.,  426.,  781.,
        962.,  141.,  146.,  696.,  803.,  153.,  553.,  636.,  976.,
        202.,  328.,  421.,  288.,   40.])

In [29]:
# 어떤구간에 속해야하나?
labels = bins.searchsorted(data)
labels

array([1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1], dtype=int64)

In [30]:
Series(data).groupby(labels).mean()

1     58.000000
2    568.731707
dtype: float64

In [32]:
# numpy에는 위 과정을 계산해주는 함수 > digitize
np.digitize(data,bins)

array([1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1], dtype=int64)

## 12.7: NumPy matrix 클래스(NumPy matrix class)
- 2차원 배열에서 하나의 로우 선택 : X[1,:]
- 2차원 배열에서 하나의 컬럼 선택 : X[:,1]

In [6]:
X =  np.array([[ 8.82768214,  3.82222409, -1.14276475,  2.04411587],
               [ 3.82222409,  6.75272284,  0.83909108,  2.08293758],
               [-1.14276475,  0.83909108,  5.01690521,  0.79573241],
               [ 2.04411587,  2.08293758,  0.79573241,  6.24095859]])
# One-dimensional(한 로우)
X[:,0]
y = X[:,:1] # 슬라이싱에 의한 2차원
X
y

array([ 8.8277,  3.8222, -1.1428,  2.0441])

array([[ 8.8277,  3.8222, -1.1428,  2.0441],
       [ 3.8222,  6.7527,  0.8391,  2.0829],
       [-1.1428,  0.8391,  5.0169,  0.7957],
       [ 2.0441,  2.0829,  0.7957,  6.241 ]])

array([[ 8.8277],
       [ 3.8222],
       [-1.1428],
       [ 2.0441]])

In [8]:
# 행렬곱 : y_transpse * X * y
np.dot(y.T, np.dot(X,y))

array([[ 1195.468]])

In [11]:
# numpy.matrix : 클래스 제공시 계산
Xm = np.matrix(X)
ym = Xm[:,0]
Xm
ym
ym.T * Xm * ym

matrix([[ 8.8277,  3.8222, -1.1428,  2.0441],
        [ 3.8222,  6.7527,  0.8391,  2.0829],
        [-1.1428,  0.8391,  5.0169,  0.7957],
        [ 2.0441,  2.0829,  0.7957,  6.241 ]])

matrix([[ 8.8277],
        [ 3.8222],
        [-1.1428],
        [ 2.0441]])

matrix([[ 1195.468]])

In [14]:
# Xm.I : matrix 클래스는 역행렬을 반환하는 특수 속성(I).
Xm.I * X

matrix([[  1.0000e+00,   0.0000e+00,  -6.9389e-17,   0.0000e+00],
        [ -1.1102e-16,   1.0000e+00,   2.7756e-17,  -2.7756e-17],
        [  1.1102e-16,   5.5511e-17,   1.0000e+00,   0.0000e+00],
        [  2.7756e-17,   2.7756e-17,  -2.7756e-17,   1.0000e+00]])

- numpy.matrix는 추천 안함.
- np.asarray : matrix로 변환하고 데이터를 복사하지 않음.

## 12.8: 고급 배열 입,출력(Advanced array input and output)

### 12.8.1: 메모리 맵 파일(Memory-mapped files)
##### memmap 
- Numpy에 ndarray와 유사한 객체
- 배열전체를 메모리에 적재하지 않고 큰 파일의 작은부분을 읽고 쓸 수 있음.
- ndarray의 대체재

In [11]:
# memmap : np.memmap 함수에 파일경로와 dtype, 모양 그리고 파일의 모드를 전달해야함.
mmap = np.memmap('mymmap2', dtype='float64', mode='w+', shape=(10000,10000))
mmap

OSError: [Errno 22] Invalid argument: 'mymmap2'

In [6]:
# memmap 객체 슬라이스는 디스크에 있는 데이터에 대한 뷰를 반환.
section = mmap[:5]

In [10]:
section[:] = np.random.randn(5,10000) # 데이터 대입
# 파일이 객체처럼 메모리에 잠시 보관(buffered)뒤 flush로 호출해서 디스크에 기록
mmap.flush()
mmap
del mmap

NameError: name 'mmap' is not defined

In [13]:
mmap = np.memmap('mymmap2', dtype='float64', mode='w+', shape=(10000,10000))
mmap

OSError: [Errno 22] Invalid argument: 'mymmap2'

### 12.8.2: HDF5 및 기타 배열 저장 옵션(HDF5 and other array storage options)
#### PyTables & h5py
- 효율적, HDF5(Hierarchical DataFormat) 형식으로 압축할 수 있도록 배열데이터를 저장가능하게끔 하는 Numpy의 친화적인 파이썬 프로젝트.
- 수백기가 혹슨 수 테라바이트의 데이터를 HDF5 형식으로 안전하게 저장가능.
- PyTables : 구조화 된 배열을 진보된 질의 기능 및 질의속도를 높일 수 있도록 컬럼 색인을 추가하는 기능과 함께 사용할 수 있음, 관계형 데이터베이스에서 제공하는 테이블 색인기능과 비슷.

## 12.9: 성능 팁(Performance Tips)

- 파이썬 반복문과 조건문을 배열연산과 boolean 배열연산으로 변환.
- 가능한 한 broadcasting을 사용.
- 배열의 뷰(slice)를 사용해 데이터를 복사하는 것을 피한다.
- ufunc 메서드를 활용.

### 12.9.1: 인접메모리의 중요성(The importance of contiguous memory)

In [17]:
arr_c = np.ones((1000,1000), order='C')
arr_f = np.ones((1000,1000), order='F')
arr_c.flags # C의 로우 우선순위
arr_f.flags # 포트란의 컬럼 우선순위
arr_f.flags.f_contiguous

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

True

In [19]:
%timeit arr_c.sum(1)
%timeit arr_f.sum(1) # 배열의 로우 합은 메모리에 로우가 연속적으로 존재하므로 이론적으로 arr_f가 빠르다.

100 loops, best of 3: 3.87 ms per loop
100 loops, best of 3: 2.31 ms per loop


In [21]:
arr_f.copy('C').flags # C나 F로 해서 복사가능.

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [23]:
arr_c[:50].flags.contiguous
arr_c[:, :50].flags

True

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [24]:
%xdel arr_c
%xdel arr_f
%cd ..

F:\Jupyternotebook\version_3.0


### 12.9.2: 기타성능옵션: Cython, f2py, C
- Cython: 정적자료형과 C로 작성된 코드를 파이썬 스타일의 코드에 끼워넣을 수 있는 기능.

In [25]:
# (예) 1차원 배열에서 각 원소의 합을 구하는 Cython 함수.
from numpy cimport ndarray, float64_t

def sum_elements(ndarray[float64_t] arr):
    cdef Py_ssize_t i, n = len(arr)
    cdef float64_t result = 0

    for i in range(n):
        result += arr[i]

    return result

SyntaxError: invalid syntax (<ipython-input-25-94a9beec4fb3>, line 2)